In [23]:
pip install requests pymongo pandas

Note: you may need to restart the kernel to use updated packages.


In [24]:

import os
from datetime import datetime, timezone
import requests
from pymongo import MongoClient, ASCENDING
from datetime import datetime, timezone

In [46]:
API_KEY = "API KEY"  
CITY    = "Pune"

MONGO_URI   = os.getenv("MONGO_URI", "mongodb://127.0.0.1:27017")
DB_NAME     = "sales_weatherinfo_db"
WEATHER_COL = "weather"   # existing collection is fine

In [47]:
def midnight_utc_naive(dt_utc: datetime) -> datetime:
    dt_utc = dt_utc.astimezone(timezone.utc)
    return datetime(dt_utc.year, dt_utc.month, dt_utc.day, 0, 0, 0, 0)

In [48]:
def ensure_indexes(col):
    # Idempotent
    col.create_index([("weather_date", ASCENDING), ("city", ASCENDING)], unique=True)
    col.create_index([("city", ASCENDING), ("weather_date", ASCENDING)])

In [49]:
def upsert_weather(doc: dict, col):
    col.update_one(
        {"weather_date": doc["weather_date"], "city": doc["city"]},
        {"$set": doc},
        upsert=True
    )

In [50]:
def fetch_weather_live(api_key: str, city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": api_key.strip(), "units": "metric"}

    try:
        r = requests.get(url, params=params, timeout=20)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error calling OpenWeather: {e}")

    # Fail loud with the exact API message (401, 404, 429, etc.)
    if not r.ok:
        try:
            msg = r.json()
        except Exception:
            msg = r.text
        raise RuntimeError(f"OpenWeather error {r.status_code}: {msg} | url={r.url}")

    data = r.json()
    return {
        "weather_date": datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0),  # BSON Date at midnight UTC
        "city": city,
        "temp_c": float(data["main"]["temp"]),
        "humidity": int(data["main"]["humidity"]),
        "description": str(data["weather"][0]["description"]),
        "fetched_at": datetime.now(timezone.utc),  # full timestamp of fetch
        "source": "openweather"
    }

In [51]:
def main():
    # Basic sanity check on key
    if not API_KEY or len(API_KEY.strip()) != 32:
        raise ValueError("OPENWEATHER API key looks invalid. Expected 32 chars.")

    client = MongoClient(MONGO_URI)
    col = client[DB_NAME][WEATHER_COL]

    ensure_indexes(col)          # safe even if collection/index exists
    doc = fetch_weather_live(API_KEY, CITY)   # live-only; raises on error
    upsert_weather(doc, col)

    print(f"Stored live weather for {doc['city']} on {doc['weather_date'].date()}.")

In [52]:
if __name__ == "__main__":
    main()

Stored live weather for Pune on 2025-09-23.
